In [1]:
import findspark
findspark.init()
findspark.find()
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
conf = pyspark.SparkConf().setAppName('appName').setMaster('local')
sc = pyspark.SparkContext.getOrCreate(conf=conf)
spark = SparkSession(sc)

from pyspark.sql import SQLContext

sc1 = SQLContext(sc)

In [2]:
from elasticsearch import helpers, Elasticsearch
import csv
es = Elasticsearch()

In [3]:
#checkinRdd = spark.read.json("D:\\BigDataProject\\yelpDataset\\checkin.json")
businessRdd = spark.read.format('csv').options(header='true', inferSchema='true').load("city_projection.csv")

In [100]:
businessRdd.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- postal_code: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- stars: double (nullable = true)
 |-- review_count: integer (nullable = true)
 |-- categories: string (nullable = true)



In [11]:
business = spark.read.json("D:\\BigDataProject\\yelpDataset\\business.json")
business.createOrReplaceTempView("Business")
businessRD = spark.sql("select business_id, name, address, city, state, postal_code, latitude, longitude, stars, review_count, categories from Business")


In [13]:
businessRD = spark.sql("select business_id, name, address, city, state, postal_code, latitude, longitude, stars, review_count, categories from Business where categories like '%Active Life' ")
businessRD.count()
#BusinessReviewDataRD = spark.sql("select * from businessRD where categories Like '%Gyms%' OR categories Like '%Martial Arts%' OR categories Like '%Dance Studios%' OR categories Like '%Yoga%' OR categories Like '%Pilates%' OR categories Like '%Swimming%' OR categories Like '%Massage%' OR categories Like '%Playground%' OR categories Like '%Soccer%' OR categories Like '%Basketball%' ")

2300

In [15]:
businessRD.createOrReplaceTempView("businessRD")

In [31]:
BusinessReviewDataRD = spark.sql("select * from businessRD where business_id != null And city != null And categories != null")

BusinessReviewDataRD = spark.sql("select * from businessRD where categories Like '%Tai%'")
BusinessReviewDataRD.count()

6

In [33]:
BusinessReviewDataRD.createOrReplaceTempView("BusinessReviewDataRD")
BusinessReviewDataRD.show()

+--------------------+--------------------+--------------------+---------------+-----+-----------+----------+------------+-----+------------+--------------------+
|         business_id|                name|             address|           city|state|postal_code|  latitude|   longitude|stars|review_count|          categories|
+--------------------+--------------------+--------------------+---------------+-----+-----------+----------+------------+-----+------------+--------------------+
|ZxplhMQQHjPvcFCKy...|Morrissette Marti...|116 N Old Litchfi...|Litchfield Park|   AZ|      85340|33.4942727|-112.3583458|  5.0|           5|Karate, Kickboxin...|
|huBUuQ8pU2Wj-VcV5...|BODY & BRAIN Yoga...|6720 N Durango Dr...|      Las Vegas|   NV|      89149|36.2834899|-115.2857143|  5.0|           9|Yoga, Tai Chi, Fi...|
|3LyroOHw871DJKFc8...|     Emanate Balance|   211 S Chestnut St|       Gastonia|   NC|      28054|35.2155917| -81.1603764|  5.0|           3|Health & Medical,...|
|gt2-1YqR2-_bVzPiy...|

In [73]:
#Search for fitness studio types in categories: eg: Gym,yoga,Taekwondo
BusinessReviewDataRD = spark.sql("select * from businessRD where categories Like '%Taekwondo%'")
df13 = BusinessReviewDataRD.withColumn("categories",
    F.when(BusinessReviewDataRD['categories'].contains("Taekwondo")  ,"Taekwondo").
    otherwise(BusinessReviewDataRD['categories']))

In [ ]:
#replace multiple categories with one studio type
df2 = BusinessReviewDataRD.withColumn("categories",
    F.when(BusinessReviewDataRD['categories'].contains("Gym")  ,"Gyms").
    otherwise(BusinessReviewDataRD['categories']))
df3 = BusinessReviewDataRD.withColumn("categories",
    F.when(BusinessReviewDataRD['categories'].contains("Yoga" or "yoga")  ,"Yoga Studio").
    otherwise(BusinessReviewDataRD['categories']))
df4 = BusinessReviewDataRD.withColumn("categories",
    F.when(BusinessReviewDataRD['categories'].contains("Dance")  ,"Dance studio").
    otherwise(BusinessReviewDataRD['categories']))
df5 = BusinessReviewDataRD.withColumn("categories",
    F.when(BusinessReviewDataRD['categories'].contains("Cardio")  ,"Cardio").
    otherwise(BusinessReviewDataRD['categories']))
df6 = BusinessReviewDataRD.withColumn("categories",
    F.when(BusinessReviewDataRD['categories'].contains("Martial")  ,"Martial Arts").
    otherwise(BusinessReviewDataRD['categories']))
df7 = BusinessReviewDataRD.withColumn("categories",
    F.when(BusinessReviewDataRD['categories'].contains("Meditation")  ,"Meditation centre").
    otherwise(BusinessReviewDataRD['categories']))
df8 = BusinessReviewDataRD.withColumn("categories",
    F.when(BusinessReviewDataRD['categories'].contains("Pilate")  ,"Pilates").
    otherwise(BusinessReviewDataRD['categories']))
df9 = BusinessReviewDataRD.withColumn("categories",
    F.when(BusinessReviewDataRD['categories'].contains("Karate")  ,"Karate").
    otherwise(BusinessReviewDataRD['categories']))
df10 = BusinessReviewDataRD.withColumn("categories",
    F.when(BusinessReviewDataRD['categories'].contains("Barre")  ,"Barre studio").
    otherwise(BusinessReviewDataRD['categories']))
df11 = BusinessReviewDataRD.withColumn("categories",
    F.when(BusinessReviewDataRD['categories'].contains("Boxing")  ,"Boxing Gym").
    otherwise(BusinessReviewDataRD['categories']))
df12 = BusinessReviewDataRD.withColumn("categories",
    F.when(BusinessReviewDataRD['categories'].contains("Aerial")  ,"Aerial Fitness").
    otherwise(BusinessReviewDataRD['categories']))

In [76]:
import functools 

def unionAll(dfs):
    return functools.reduce(lambda df1,df2: df1.union(df2.select(df1.columns)), dfs)

unioned_df = unionAll([df2, df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13])
unioned_df.show()

+--------------------+--------------------+--------------------+---------+-----+-----------+-------------+--------------+-----+------------+----------+
|         business_id|                name|             address|     city|state|postal_code|     latitude|     longitude|stars|review_count|categories|
+--------------------+--------------------+--------------------+---------+-----+-----------+-------------+--------------+-----+------------+----------+
|yATXrlGH3eVTGwRAD...|Relentless Power Gym|5816 W San Miguel...| Glendale|   AZ|      85301|   33.5200766|  -112.1843317|  4.0|           4|      Gyms|
|OpwRrBS1WpoxUvfi-...|F45 Training Libe...|171 E Liberty Str...|  Toronto|   ON|    M6K 3E7|   43.6381221|   -79.4181561|  4.0|           4|      Gyms|
|3fdtp-bzoE4ZgTakk...|   City of Las Vegas|       495 S Main St|Las Vegas|   NV|      89101|   36.1674401|  -115.1484346|  4.0|         202|      Gyms|
|5hEZR_myofJh3j-YF...| Youfit Health Clubs|870 E Williams Fi...|  Gilbert|   AZ|      85

In [83]:
#unioned_df1 = unioned_df.select('city','categories','state')
unioned_df.repartition(1).write.option("header", "true").csv("business.csv")

In [88]:
review = spark.read.json("D:\\BigDataProject\\yelpDataset\\review.json")

In [91]:
unioned_df.createOrReplaceTempView("unioned_df")
businessrddf = spark.sql("select business_id, name, address, city, state, postal_code, latitude, longitude, stars AS avg_stars, review_count, categories from unioned_df")

In [93]:
joinedRdd = businessrddf.join(review, 'business_id')
#Save business and review joined data into csv
joinedRdd.repartition(1).write.option("header", "true").csv("businessReview.csv")

In [99]:
joinedRdd = spark.read.format('csv').options(header='true', inferSchema='true').load("businessReview.csv")
joinedRdd.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- postal_code: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- avg_stars: string (nullable = true)
 |-- review_count: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- cool: string (nullable = true)
 |-- date: string (nullable = true)
 |-- funny: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: string (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: string (nullable = true)
 |-- user_id: string (nullable = true)



In [116]:
#change date field to timestamp for querying in elastic search
from pyspark.sql.functions import to_timestamp
from pyspark.sql.functions import col
#timeRdd = joinedRdd.withColumn("date", to_timestamp('date', "yyyy-MM-ddTHH-mm-ss"))
timeRdd = joinedRdd.withColumn('date', col('date').cast('timestamp'))

In [117]:
timeRdd.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- postal_code: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- avg_stars: string (nullable = true)
 |-- review_count: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- cool: string (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- funny: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: string (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: string (nullable = true)
 |-- user_id: string (nullable = true)



In [118]:
timeRdd.show()

+--------------------+--------------------+--------------------+--------------------+-----+-----------+----------+------------+---------+------------+------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|         business_id|                name|             address|                city|state|postal_code|  latitude|   longitude|avg_stars|review_count|  categories|cool|               date|funny|           review_id|stars|                text|useful|             user_id|
+--------------------+--------------------+--------------------+--------------------+-----+-----------+----------+------------+---------+------------+------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|QlQ7qG7Hxk_SseGnr...|Groupo Capoeira B...|  965 E Guadalupe Rd|               Tempe|   AZ|      85283|33.3636713|-111.9266971|      5.0|           5|Martial Arts|   2|2011-02-20 19:32:15

In [122]:
#load businessReview.csv into elastic search than business.csv into ES
#we are creating two index for generating business related metrics as well as review/rating related index
with open('business.csv',encoding='utf8') as f:
    index_name = 'business_details_final'
    doctype = 'cat'
    reader = csv.reader(f)
    headers = []
    index = 0
    es.indices.delete(index=index_name, ignore=[400, 404])
    es.indices.create(index=index_name, ignore=400)
    es.indices.put_mapping(
        index=index_name,
        doc_type=doctype,
        ignore=400,
        body={
            doctype: {
                "properties": {
                    "city": {
                        "type": "string"
                        },
                    "state": {
                      "type": "string"
                        }
                }
            }
        }
    )
    for row in reader:
        try:
            if(index == 0):
                  headers = row
            else:
                row[8] = float(row[8])
                #row[15] = float(row[15])
                obj = {}
                print(row)
                for i, val in enumerate(row):
                    #print(type(val))
                    obj[headers[i]] = val
                print(obj)
                  # put document into elastic search
                es.index(index=index_name, doc_type=doctype, body=obj)
                #print(obj)

        except Exception as e:
            print('error: ' + str(e) + ' in' + str(index))
        index = index + 1
    
f.close()

['yATXrlGH3eVTGwRADpDvyA', 'Relentless Power Gym', '5816 W San Miguel Ave', 'Glendale', 'AZ', '85301', '33.5200766', '-112.1843317', 4.0, '4', 'Gyms']
{'business_id': 'yATXrlGH3eVTGwRADpDvyA', 'name': 'Relentless Power Gym', 'address': '5816 W San Miguel Ave', 'city': 'Glendale', 'state': 'AZ', 'postal_code': '85301', 'latitude': '33.5200766', 'longitude': '-112.1843317', 'stars': 4.0, 'review_count': '4', 'categories': 'Gyms'}
['OpwRrBS1WpoxUvfi--bHzg', 'F45 Training Liberty Village', '171 E Liberty Street, Suite 124', 'Toronto', 'ON', 'M6K 3E7', '43.6381221', '-79.4181561', 4.0, '4', 'Gyms']
{'business_id': 'OpwRrBS1WpoxUvfi--bHzg', 'name': 'F45 Training Liberty Village', 'address': '171 E Liberty Street, Suite 124', 'city': 'Toronto', 'state': 'ON', 'postal_code': 'M6K 3E7', 'latitude': '43.6381221', 'longitude': '-79.4181561', 'stars': 4.0, 'review_count': '4', 'categories': 'Gyms'}
['3fdtp-bzoE4ZgTakkcEBzQ', 'City of Las Vegas', '495 S Main St', 'Las Vegas', 'NV', '89101', '36.167

['bvae_rjb6WcLL9odInjPng', 'Functional Integrated Training', '5380 King James Way, Ste A1', 'Fitchburg', 'WI', '53719', '43.0174466', '-89.472961', 5.0, '7', 'Gyms']
{'business_id': 'bvae_rjb6WcLL9odInjPng', 'name': 'Functional Integrated Training', 'address': '5380 King James Way, Ste A1', 'city': 'Fitchburg', 'state': 'WI', 'postal_code': '53719', 'latitude': '43.0174466', 'longitude': '-89.472961', 'stars': 5.0, 'review_count': '7', 'categories': 'Gyms'}
['79zzolCRbeM_1YjnssWBjw', 'Van Hook Fitness and Sports Performance', '900 W Warm Springs Rd, Ste 103', 'Henderson', 'NV', '89011', '36.05680286', '-115.0249805', 5.0, '11', 'Gyms']
{'business_id': '79zzolCRbeM_1YjnssWBjw', 'name': 'Van Hook Fitness and Sports Performance', 'address': '900 W Warm Springs Rd, Ste 103', 'city': 'Henderson', 'state': 'NV', 'postal_code': '89011', 'latitude': '36.05680286', 'longitude': '-115.0249805', 'stars': 5.0, 'review_count': '11', 'categories': 'Gyms'}
['edxPQjfsnHRzUeh3XsQSMw', 'Planet Fitness -

['airZTbkgq9dUb-RAt_H0FQ', 'The Body Shop', '2432 W Peoria Ave, Ste 1242', 'Phoenix', 'AZ', '85029', '33.5824149', '-112.1117315', 5.0, '8', 'Gyms']
{'business_id': 'airZTbkgq9dUb-RAt_H0FQ', 'name': 'The Body Shop', 'address': '2432 W Peoria Ave, Ste 1242', 'city': 'Phoenix', 'state': 'AZ', 'postal_code': '85029', 'latitude': '33.5824149', 'longitude': '-112.1117315', 'stars': 5.0, 'review_count': '8', 'categories': 'Gyms'}
['_KUzBWCoswmDpFat5DIJSw', 'Orangetheory Fitness Scottsdale Ranch', '9301 E Shea Blvd, Ste 114-116', 'Scottsdale', 'AZ', '85260', '33.5809114', '-111.8811427', 4.5, '50', 'Gyms']
{'business_id': '_KUzBWCoswmDpFat5DIJSw', 'name': 'Orangetheory Fitness Scottsdale Ranch', 'address': '9301 E Shea Blvd, Ste 114-116', 'city': 'Scottsdale', 'state': 'AZ', 'postal_code': '85260', 'latitude': '33.5809114', 'longitude': '-111.8811427', 'stars': 4.5, 'review_count': '50', 'categories': 'Gyms'}
['T20oR2Z-0AyyPNs9PmPcsw', 'Anytime Fitness', '1222 1st St SW', 'Calgary', 'AB', 'T2

['Z9PzicyQuvzUazASvRxtg', 'Excel Defense Studios', '5803 W Craig Rd, Ste 102', 'Las Vegas', 'NV', '89130', '36.2387598', '-115.2241644', 4.0, '7', 'Gyms']
{'business_id': 'Z9PzicyQuvzUazASvRxtg', 'name': 'Excel Defense Studios', 'address': '5803 W Craig Rd, Ste 102', 'city': 'Las Vegas', 'state': 'NV', 'postal_code': '89130', 'latitude': '36.2387598', 'longitude': '-115.2241644', 'stars': 4.0, 'review_count': '7', 'categories': 'Gyms'}
['pDn3TAjVcY7TqtZD9YI38A', 'Psycle', '24945 Detroit Rd', 'Westlake', 'OH', '44145', '41.470266', '-81.896853', 5.0, '4', 'Gyms']
{'business_id': 'pDn3TAjVcY7TqtZD9YI38A', 'name': 'Psycle', 'address': '24945 Detroit Rd', 'city': 'Westlake', 'state': 'OH', 'postal_code': '44145', 'latitude': '41.470266', 'longitude': '-81.896853', 'stars': 5.0, 'review_count': '4', 'categories': 'Gyms'}
['so3mxlBchivcXS9UgyE4gA', 'EōS Fitness - Las Vegas: Southeast/Stephanie', '35 S Stephanie St', 'Henderson', 'NV', '89012', '36.0277488', '-115.0499692', 3.5, '224', 'Gyms'

['LMAgC-azX2BOnX3siaT0hw', 'Desert Lights Gymnastics', '6175 W Detroit St', 'Chandler', 'AZ', '85226', '33.3079302', '-111.9495011', 5.0, '8', 'Gyms']
{'business_id': 'LMAgC-azX2BOnX3siaT0hw', 'name': 'Desert Lights Gymnastics', 'address': '6175 W Detroit St', 'city': 'Chandler', 'state': 'AZ', 'postal_code': '85226', 'latitude': '33.3079302', 'longitude': '-111.9495011', 'stars': 5.0, 'review_count': '8', 'categories': 'Gyms'}
['#NAME?', 'Dynamic Fitness Solutions', '8535 Tanglewood Sq', 'Chagrin Falls', 'OH', '44023', '41.389879', '-81.337407', 4.0, '4', 'Gyms']
{'business_id': '#NAME?', 'name': 'Dynamic Fitness Solutions', 'address': '8535 Tanglewood Sq', 'city': 'Chagrin Falls', 'state': 'OH', 'postal_code': '44023', 'latitude': '41.389879', 'longitude': '-81.337407', 'stars': 4.0, 'review_count': '4', 'categories': 'Gyms'}
['QPlA2f7ewUA6gzWb_kO6FA', 'Xtreme Couture MMA', '4055 W Sunset Rd', 'Las Vegas', 'NV', '89118', '36.071536', '-115.1937517', 5.0, '29', 'Gyms']
{'business_id':

...

['ZughgAjbQIYn1OZyCA69PQ', 'Sculpt and Sweat', '9720 W Tropicana Ave, Ste 120', 'Las Vegas', 'NV', '89147', '36.1009248', '-115.3048697', 5.0, '8', 'Pilates']
{'business_id': 'ZughgAjbQIYn1OZyCA69PQ', 'name': 'Sculpt and Sweat', 'address': '9720 W Tropicana Ave, Ste 120', 'city': 'Las Vegas', 'state': 'NV', 'postal_code': '89147', 'latitude': '36.1009248', 'longitude': '-115.3048697', 'stars': 5.0, 'review_count': '8', 'categories': 'Pilates'}
['STEm_-WIauJ8hC0cZbX-7Q', 'Hilliard Studio Method', '516 Fenton Pl', 'Charlotte', 'NC', '28207', '35.20105427', '-80.82373457', 4.0, '10', 'Pilates']
{'business_id': 'STEm_-WIauJ8hC0cZbX-7Q', 'name': 'Hilliard Studio Method', 'address': '516 Fenton Pl', 'city': 'Charlotte', 'state': 'NC', 'postal_code': '28207', 'latitude': '35.20105427', 'longitude': '-80.82373457', 'stars': 4.0, 'review_count': '10', 'categories': 'Pilates'}
['kcF4zNo7dLECwHml9GOv_A', 'Barre Body Studio', '1202 20th Avenue SE', 'Calgary', 'AB', 'T2G 3G2', '51.035672', '-114.03

['z3baEfDSwTMk6q2bHuqEsw', 'Omni Health & Fitness Center', '2685 Celanese Rd', 'Rock Hill', 'SC', '29732', '34.9744969', '-81.023194', 3.0, '5', 'Pilates']
{'business_id': 'z3baEfDSwTMk6q2bHuqEsw', 'name': 'Omni Health & Fitness Center', 'address': '2685 Celanese Rd', 'city': 'Rock Hill', 'state': 'SC', 'postal_code': '29732', 'latitude': '34.9744969', 'longitude': '-81.023194', 'stars': 3.0, 'review_count': '5', 'categories': 'Pilates'}
['pUjnlYxbRruEfamFMkQvog', 'Arizona Wado Karate', '1992 N Alma School Rd', 'Chandler', 'AZ', '85224', '33.334148', '-111.860794', 5.0, '4', 'Karate']
{'business_id': 'pUjnlYxbRruEfamFMkQvog', 'name': 'Arizona Wado Karate', 'address': '1992 N Alma School Rd', 'city': 'Chandler', 'state': 'AZ', 'postal_code': '85224', 'latitude': '33.334148', 'longitude': '-111.860794', 'stars': 5.0, 'review_count': '4', 'categories': 'Karate'}
['ZiAruTCyaFUOt8IkOik-ig', 'No Limits Karate Academy', '747 W Pinnacle Peak, Ste A101', 'Phoenix', 'AZ', '85027', '33.6982157', 

['k-YKRajhSb6fi6czwhGRGg', 'StudioMixx', '7025 E 5th Ave', 'Scottsdale', 'AZ', '85251', '33.4972471', '-111.9299454', 4.5, '30', 'Barre studio']
{'business_id': 'k-YKRajhSb6fi6czwhGRGg', 'name': 'StudioMixx', 'address': '7025 E 5th Ave', 'city': 'Scottsdale', 'state': 'AZ', 'postal_code': '85251', 'latitude': '33.4972471', 'longitude': '-111.9299454', 'stars': 4.5, 'review_count': '30', 'categories': 'Barre studio'}
['xA96fu69t3M1rbU9KCAqMg', 'TenPoint5 Barre Studio - Chandler', '7131 W Ray Rd, Ste 40', 'Chandler', 'AZ', '85226', '33.3181459', '-111.9657564', 4.5, '15', 'Barre studio']
{'business_id': 'xA96fu69t3M1rbU9KCAqMg', 'name': 'TenPoint5 Barre Studio - Chandler', 'address': '7131 W Ray Rd, Ste 40', 'city': 'Chandler', 'state': 'AZ', 'postal_code': '85226', 'latitude': '33.3181459', 'longitude': '-111.9657564', 'stars': 4.5, 'review_count': '15', 'categories': 'Barre studio'}
['BiGJEC4WxcoW578wi1UBMw', 'Pure Barre - Pittsburgh Mt. Lebanon', '1612 Cochran Rd', 'Pittsburgh', 'PA',

['MKahXDTretOWrB-xD4UGDw', 'Shape Training', '5877 Commerce St', 'Pittsburgh', 'PA', '15206', '40.45950942', '-79.92856068', 5.0, '11', 'Boxing Gym']
{'business_id': 'MKahXDTretOWrB-xD4UGDw', 'name': 'Shape Training', 'address': '5877 Commerce St', 'city': 'Pittsburgh', 'state': 'PA', 'postal_code': '15206', 'latitude': '40.45950942', 'longitude': '-79.92856068', 'stars': 5.0, 'review_count': '11', 'categories': 'Boxing Gym'}
['wG13iXDU6cJAZeay5w4nkg', 'Studio KO', '788 King Street W', 'Toronto', 'ON', 'M5V 1N6', '43.64317', '-79.4066254', 4.0, '6', 'Boxing Gym']
{'business_id': 'wG13iXDU6cJAZeay5w4nkg', 'name': 'Studio KO', 'address': '788 King Street W', 'city': 'Toronto', 'state': 'ON', 'postal_code': 'M5V 1N6', 'latitude': '43.64317', 'longitude': '-79.4066254', 'stars': 4.0, 'review_count': '6', 'categories': 'Boxing Gym'}
['tzI0EDRYlaw4GmVERftKhg', 'Rival Boxing Gear USA', '1335 E Sunset Rd, Ste G', 'Las Vegas', 'NV', '89119', '36.0712738', '-115.1358324', 5.0, '4', 'Boxing Gym']

['hBZxyhnmQ3QSmCarG3QZNA', 'Arizona Balloons', '21705 N 19th Ave', 'Phoenix', 'AZ', '85027', '33.68274593', '-112.0989336', 5.0, '8', 'Aerial Fitness']
{'business_id': 'hBZxyhnmQ3QSmCarG3QZNA', 'name': 'Arizona Balloons', 'address': '21705 N 19th Ave', 'city': 'Phoenix', 'state': 'AZ', 'postal_code': '85027', 'latitude': '33.68274593', 'longitude': '-112.0989336', 'stars': 5.0, 'review_count': '8', 'categories': 'Aerial Fitness'}
['hjsJ4P3gwpiEcCdRQc7_Ag', 'Vertical Fix', '7425 S Harl Ave, Ste 3', 'Tempe', 'AZ', '85283', '33.351308', '-111.965949', 4.5, '19', 'Aerial Fitness']
{'business_id': 'hjsJ4P3gwpiEcCdRQc7_Ag', 'name': 'Vertical Fix', 'address': '7425 S Harl Ave, Ste 3', 'city': 'Tempe', 'state': 'AZ', 'postal_code': '85283', 'latitude': '33.351308', 'longitude': '-111.965949', 'stars': 4.5, 'review_count': '19', 'categories': 'Aerial Fitness'}
['pInDxskIAcXxbQ5lDI-_ZQ', 'Rainbow Ryders Hot Air Balloon Ride', '715 East Covey Ln', 'Phoenix', 'AZ', '85024', '33.67815641', '-112.06